# Minist 数据集测试

> 范例来自于 https://www.tensorflow.org/tutorials/quickstart/beginner

## 准备工作

- 引入必要的包

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random as rdm
from os import path, curdir, makedirs

- 定义保存和读取模型的函数

In [ ]:
# 保存模型
def save_weights(model: tf.keras.Model, model_path: str, model_file='model') -> None:
    model_path = path.abspath(path.join(curdir, model_path))
    if not path.exists(model_path):
        makedirs(model_path, exist_ok=True)

    model_path = path.join(model_path, model_file)
    model.save_weights(model_path)


# 读取模型
def load_weights(model: tf.keras.Model, model_path: str, model_file='model') -> bool:
    model_path = path.abspath(path.join(curdir, model_path))
    if not path.exists(model_path):
        return False

    model_path = path.join(model_path, model_file)
    if not path.exists('{}.index'.format(model_path)):
        return False

    model.load_weights(model_path)
    return True

- 定义显示图片的函数

In [ ]:
def show_image(img: np.ndarray, *, figsize=(1, 1), cmap='gray') -> None:
    plt.figure(figsize=figsize)
    if len(img.shape) > 2:
        img = img.reshape(img.shape[1:])

    plt.imshow(img, cmap=cmap)
    plt.show()

## 手写数字识别

### 读取数据集

In [ ]:
handwriting_mnist = tf.keras.datasets.mnist
(x_train_hw, y_train_hw), (x_test_hw, y_test_hw) = handwriting_mnist.load_data()

x_train_hw, x_test_hw = x_train_hw / 255.0, x_test_hw / 255.0  # 归一化
print('* train and test data loaded, shape is: {}'.format(x_train_hw.shape))

### 定义预测函数

In [ ]:
def handwriting_predict(model: tf.keras.Model) -> None:
    index = rdm.randint(0, len(x_test_hw))
    print('* random index is {}'.format(index))

    img = np.array([x_test_hw[index]])
    show_image(img.reshape(28, 28))

    result = tf.argmax(model.predict(img)[0])
    print('* result number is: {}'.format(result))

### 仅通过 softmax 构建网络

- 定义模型

In [ ]:
tf.keras.backend.clear_session()

model_hw_softmax = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # 输入层，输入 28 点阵图片
    tf.keras.layers.Dropout(0.2),  # dropout 正则化
    tf.keras.layers.Dense(10, activation='softmax')  # 10 单元的 softmax 层作为输出
])
print(model_hw_softmax.summary())

# 编译模型。指定优化函数，损失函数以及度量函数
model_hw_softmax.compile(optimizer='adam',
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])
print('* compile finished')

- 训练模型

In [ ]:
model_path = 'models/mnist_num_softmax'
if not load_weights(model_hw_softmax, model_path=model_path):
    model_hw_softmax.fit(x_train_hw, y_train_hw, epochs=5)
    save_weights(model_hw_softmax, model_path=model_path)
    
print('* triainning completed')

- 测试模型

In [ ]:
lost, accuracy = model_hw_softmax.evaluate(x_test_hw, y_test_hw, verbose=2)
print('* finish test, lost is {}, accuracy is {}'.format(lost, accuracy))

- 应用模型

In [ ]:
handwriting_predict(model_hw_softmax)

### 加入隐层

- 定义模型

In [ ]:
tf.keras.backend.clear_session()

model_hw_relu = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # 输入层，输入 28 点阵图片
    tf.keras.layers.Dense(128, activation='relu'),  # 128 神经元的全连接层
    tf.keras.layers.Dropout(0.2),  # dropout 正则化
    tf.keras.layers.Dense(10, activation='softmax')  # 10 单元的 softmax 层作为输出
])
print(model_hw_relu.summary())

# 编译模型
model_hw_relu.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
print('* compile finished')

- 训练模型

In [ ]:
model_path = 'models/mnist_num_relu'
if not load_weights(model_hw_relu, model_path=model_path):
    model_hw_relu.fit(x_train_hw, y_train_hw, epochs=5)
    save_weights(model_hw_relu, model_path=model_path)
    
print('* trainning completed')

- 测试模型

In [ ]:
lost, accuracy = model_hw_relu.evaluate(x_test_hw, y_test_hw, verbose=2)
print('* finish test, lost is {}, accuracy is {}'.format(lost, accuracy))

- 应用模型

In [ ]:
handwriting_predict(model_hw_relu)

## 服装类别识别

> 范例来源于 https://www.tensorflow.org/tutorials/keras/classification¶

- 定义服装类别

In [ ]:
fashion_class_names = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

### 读取数据集

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train_f, y_train_f), (x_test_f, y_test_f) = fashion_mnist.load_data()

x_train_f, x_test_f = x_train_f / 255.0, x_test_f / 255.0
print('* train and test data loaded, shape is: {}'.format(x_train_f.shape))

### 定义预测函数

In [ ]:
def fashion_predict(model: tf.keras.Model, dataset=x_test_f) -> None:
    index = rdm.randint(0, len(dataset))
    print('* random index is {}'.format(index))

    img = np.array([dataset[index]])  # select random image from test list
    show_image(img.reshape((28, 28)))

    result = np.argmax(model.predict(img)[0])
    class_name = fashion_class_names[result]
    print('* result is "{}"'.format(class_name))

### 通过隐层和 softmax 构建网络

- 定义模型

In [ ]:
tf.keras.backend.clear_session()

model_f_relu = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # 输入层，输入 28 点阵图片
    tf.keras.layers.Dense(128, activation='relu'),  # 128 神经元的全连接层
    tf.keras.layers.Dropout(0.2),  # dropout 正则化
    tf.keras.layers.Dense(10, activation='softmax')  # 10 单元的 softmax 层作为输出
])
print(model_f_relu.summary())

model_f_relu.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
print('* compile finished')

- 训练模型

In [ ]:
model_path = 'models/mnist_fashion_dense'
if not load_weights(model_f_relu, model_path=model_path):
    model_f_relu.fit(x_train_f, y_train_f, epochs=5)
    save_weights(model_f_relu, model_path)
    
print('* trainning completed')

- 测试模型

In [ ]:
lost, accuracy = model_f_relu.evaluate(x_test_f, y_test_f, verbose=2)
print('* finish test, lost is {}, accuracy is {}'.format(lost, accuracy))

- 应用模型

In [ ]:
fashion_predict(model_f_relu)

### 通过 CNN 构建网络

- 重构训练数据

In [ ]:
# 增加颜色纬度，单色
x_train_fcnn, x_test_fcnn = x_train_f.reshape(-1, 28, 28, 1), x_test_f.reshape(-1, 28, 28, 1)

- 定义模型

In [ ]:
tf.keras.backend.clear_session()

model_f_cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # 输入层，输入 28 点阵图片
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),  # dropout 正则化
    tf.keras.layers.Flatten(),  # 平面化
    tf.keras.layers.Dense(128, activation='relu'),  # 128 神经元的全连接层
    tf.keras.layers.Dropout(0.5),  # dropout 正则化
    tf.keras.layers.Dense(10, activation='softmax')  # 10 单元的 softmax 层作为输出
])
print(model_f_cnn.summary())

model_f_cnn.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
print('* compile finished')

- 训练模型

In [ ]:
model_path = 'models/mnist_fashion_cnn'
if not load_weights(model_f_cnn, model_path=model_path):
    model_f_cnn.fit(x_train_fcnn, y_train_f, epochs=5)
    save_weights(model_f_cnn, model_path=model_path)

- 测试模型

In [ ]:
lost, accuracy = model_f_cnn.evaluate(x_test_fcnn, y_test_f, verbose=2)
print('* finish test, lost is {}, accuracy is {}'.format(lost, accuracy))

- 应用模型

In [ ]:
fashion_predict(model_f_cnn, dataset=x_test_fcnn)